# Using Pandas Reporter for Analyzing The ACS

In this example, we'll use Census Reporter and pandasreporter to combine age groups in a census table and report aggregates and ratios. To keep the extent of the data small, we'll analyze COunty subdivisions of Bexar County Texas. 

To start the anslysis, visit [Census Reporter](https://censusreporter.org) and search for "Bexar" in the "Profile" search box. Select "Bexar County, Texas". On the profile page, in the 'Find data for this place' search box, search for "Poverty Status by Sex By Age." You should get a hit for Table B17001, "Poverty Status in the Past 12 Months by Sex by Age"

On the data page for [Table B17001](https://censusreporter.org/data/table/?table=B17001&geo_ids=05000US48029&primary_geo_id=05000US48029) select, in the left margin, under "Divide Bexar County, TX into " the link for "county subdivisions". You'll end up at [a page with columns for each of the county subdivisions](https://censusreporter.org/data/table/?table=B17001&geo_ids=05000US48029,060|05000US48029&primary_geo_id=05000US48029)

Look in the URL bar, and you'll see at the end of the url, these URL quiery parameters:  `geo_ids=05000US48029,060` and `primary_geo_id=05000US48029`. The `primary_geo_id` is the geoid for Bexar County, Tx. You can also get this value from the profile page. For instance the profile page for the Cit of San Diego has the URL https://censusreporter.org/profiles/16000US0666000-san-diego-ca/, where `16000US0666000` is the geoid for San Diego. 

The `,060` part of `geoids` is the summary level for the county subdivisions. You can also get the summar level code [from this list](https://www.census.gov/geo/maps-data/data/summary_level.html). Furthermore, the first three digits of any geoid is the summar levels, so for the place San Diego, with geoid `16000US0666000`, the summary level code is `160`. 

Now you have everything you need to fetch a table with the Census Reporter API: we need the table id, the geoid of the containing geography, and the summary level code for the divisons of the containing geography.

* Table: `B17001` ( Poverty Status in the Past 12 Months by Sex by Age )
* Containing geography geoid: `05000US48029` ( Bexar County,  Tx )
* Summary level: `060` ( County Subdivisions )



In [1]:
import pandas as pd
import numpy as np
import pandasreporter

df = pandasreporter.get_dataframe('B17001', '140',  '05000US48029', cache=True)
df.head(2)

,geoid,name,B17001001,B17001001_m90,B17001002,B17001002_m90,B17001003,B17001003_m90,B17001004,B17001004_m90,...,B17001055,B17001055_m90,B17001056,B17001056_m90,B17001057,B17001057_m90,B17001058,B17001058_m90,B17001059,B17001059_m90
0,14000US48029160901,"Census Tract 1609.01, Bexar, TX",3092.0,405.0,1092.0,419.0,523.0,239.0,101.0,82.0,...,135.0,47.0,142.0,56.0,148.0,55.0,64.0,28.0,63.0,40.0
1,14000US48029180201,"Census Tract 1802.01, Bexar, TX",4517.0,433.0,1276.0,348.0,673.0,234.0,71.0,74.0,...,244.0,96.0,186.0,74.0,146.0,59.0,132.0,52.0,91.0,46.0


The `get_dataframe` returns a dataframe, but the columns have cryptic codes. You can get a new view with different column name that are easier to understand. The access methods are: 

* `df.titled_columns` for human readable titles
* `df.coded_columns` to return to the codes
* `df.ct_columns` for a combination of codes and titles. This one is the easiest to use


In [2]:
df = df.ct_columns
df.head(2)

,geoid,name,B17001001 Total,Margins for B17001001 Total,B17001002 Total Income in the past 12 months below poverty level,Margins for B17001002 Total Income in the past 12 months below poverty level,B17001003 Total Income in the past 12 months below poverty level Male,Margins for B17001003 Total Income in the past 12 months below poverty level Male,B17001004 Total Income in the past 12 months below poverty level Male Under 5 years,Margins for B17001004 Total Income in the past 12 months below poverty level Male Under 5 years,...,B17001055 Total Income in the past 12 months at or above poverty level Female 35 to 44 years,Margins for B17001055 Total Income in the past 12 months at or above poverty level Female 35 to 44 years,B17001056 Total Income in the past 12 months at or above poverty level Female 45 to 54 years,Margins for B17001056 Total Income in the past 12 months at or above poverty level Female 45 to 54 years,B17001057 Total Income in the past 12 months at or above poverty level Female 55 to 64 years,Margins for B17001057 Total Income in the past 12 months at or above poverty level Female 55 to 64 years,B17001058 Total Income in the past 12 months at or above poverty level Female 65 to 74 years,Margins for B17001058 Total Income in the past 12 months at or above poverty level Female 65 to 74 years,B17001059 Total Income in the past 12 months at or above poverty level Female 75 years and over,Margins for B17001059 Total Income in the past 12 months at or above poverty level Female 75 years and over
0,14000US48029160901,"Census Tract 1609.01, Bexar, TX",3092.0,405.0,1092.0,419.0,523.0,239.0,101.0,82.0,...,135.0,47.0,142.0,56.0,148.0,55.0,64.0,28.0,63.0,40.0
1,14000US48029180201,"Census Tract 1802.01, Bexar, TX",4517.0,433.0,1276.0,348.0,673.0,234.0,71.0,74.0,...,244.0,96.0,186.0,74.0,146.0,59.0,132.0,52.0,91.0,46.0


Regardless of how you set the column names, you can always index with the codes, or the last three digits of the code. Additionally, for a lot of the special math functions, you can use a string with the last three digits. 

In [3]:
assert np.round(df['B17001001'].mean(),4) == 4893.3689
assert np.round(df['001'].mean(),4) == 4893.3689

In [4]:
# Here is a nice view of all of the column names, with some example columns
df.iloc[:3].T

,0,1,2
geoid,14000US48029160901,14000US48029180201,14000US48029151301
name,0,1,2
B17001001 Total,3092,4517,5874
Margins for B17001001 Total,405,433,607
B17001002 Total Income in the past 12 months below poverty level,1092,1276,1281
Margins for B17001002 Total Income in the past 12 months below poverty level,419,348,497
B17001003 Total Income in the past 12 months below poverty level Male,523,673,611
Margins for B17001003 Total Income in the past 12 months below poverty level Male,239,234,260
B17001004 Total Income in the past 12 months below poverty level Male Under 5 years,101,71,33
Margins for B17001004 Total Income in the past 12 months below poverty level Male Under 5 years,82,74,38


In [5]:
# Or, just dump the columns
[e for e in df.columns if '12 to 14 years' in str(e)]

['B17001007 Total Income in the past 12 months below poverty level Male 12 to 14 years',
 'Margins for B17001007 Total Income in the past 12 months below poverty level Male 12 to 14 years',
 'B17001021 Total Income in the past 12 months below poverty level Female 12 to 14 years',
 'Margins for B17001021 Total Income in the past 12 months below poverty level Female 12 to 14 years',
 'B17001036 Total Income in the past 12 months at or above poverty level Male 12 to 14 years',
 'Margins for B17001036 Total Income in the past 12 months at or above poverty level Male 12 to 14 years',
 'B17001050 Total Income in the past 12 months at or above poverty level Female 12 to 14 years',
 'Margins for B17001050 Total Income in the past 12 months at or above poverty level Female 12 to 14 years']

In [6]:
# Values for Young males 12 to 17
sumsdf = pandasreporter.CensusDataFrame()
sumsdf['group'] = df.geoid.str[-1] 
sumsdf['m1217'], sumsdf['m1217_m90'] =  df.sum_m('B17001007', 'B17001008', 'B17001009')
sumsdf.head(4)

,group,m1217,m1217_m90
0,1,41.0,32.449961
1,1,127.0,73.280284
2,1,88.0,62.136946
3,2,44.0,50.467812


In [7]:
# Add in total males and total females
sumsdf['males'], sumsdf['males_m90'] = df['B17001003'], df['B17001003_m90']
sumsdf['females'], sumsdf['females_m90'] = df['B17001017'], df['B17001017_m90']
sumsdf.head(4)

,group,m1217,m1217_m90,males,males_m90,females,females_m90
0,1,41.0,32.449961,523.0,239.0,569.0,211.0
1,1,127.0,73.280284,673.0,234.0,603.0,198.0
2,1,88.0,62.136946,611.0,260.0,670.0,283.0
3,2,44.0,50.467812,241.0,145.0,381.0,178.0


In [8]:
# proportion of young males. This is a proportion because young males is a subset of all males
sumsdf['prop_m1217'], sumsdf['prop_m1217_m90'] = sumsdf.proportion('m1217', 'males')
sumsdf.head(4)

,group,m1217,m1217_m90,males,males_m90,females,females_m90,prop_m1217,prop_m1217_m90
0,1,41.0,32.449961,523.0,239.0,569.0,211.0,0.078394,0.050659
1,1,127.0,73.280284,673.0,234.0,603.0,198.0,0.188707,0.086897
2,1,88.0,62.136946,611.0,260.0,670.0,283.0,0.144026,0.081155
3,2,44.0,50.467812,241.0,145.0,381.0,178.0,0.182573,0.178287


In [9]:
# Ratio of male to female. Since one is not a subset of the other, use a ratio. 
sumsdf['fm'], sumsdf['fm_m90'] = sumsdf.ratio('females', 'males')
sumsdf.add_rse('fm') # Add a relative std err column

sumsdf.head(4)

,group,m1217,m1217_m90,males,males_m90,females,females_m90,prop_m1217,prop_m1217_m90,fm,fm_m90,fm_rse
0,1,41.0,32.449961,523.0,239.0,569.0,211.0,0.078394,0.050659,1.087954,0.640270,35.775564
1,1,127.0,73.280284,673.0,234.0,603.0,198.0,0.188707,0.086897,0.895988,0.428496,29.072260
2,1,88.0,62.136946,611.0,260.0,670.0,283.0,0.144026,0.081155,1.096563,0.657471,36.448273
3,2,44.0,50.467812,241.0,145.0,381.0,178.0,0.182573,0.178287,1.580913,1.204260,46.306966


In [10]:
# Group and sum. The sum() method will summ the _m90 columns as root of summed squares. 
# The sort is required because the aggregation screws up the column order. 

# NaNs screw up the calculations, although not sure that using zero is sensible. 
sumsdf.prop_m1217_m90.fillna(0, inplace = True)
sumsdf.fm_m90.fillna(0, inplace = True)

ss = sumsdf.groupby('group').sum()
# ss[sorted(ss.columns)] One way to do it, or:
ss.reindex_axis(sorted(ss.columns), axis=1)

,females,females_m90,fm,fm_m90,fm_rse,m1217,m1217_m90,males,males_m90,prop_m1217,prop_m1217_m90
,sum,sum_rs,sum,sum_rs,sum,sum,sum_rs,sum,sum_rs,sum,sum_rs
group,,,,,,,,,,,
0,62285.0,2556.847277,142.533792,24.041011,4671.629026,6089.0,632.922586,52771.0,2308.309555,12.201799,3.067930
1,26366.0,1762.735658,73.070193,10.284589,2748.636381,2930.0,434.650434,22326.0,1558.128044,7.635554,4.163609
2,27288.0,1929.867353,74.661264,11.165912,inf,2645.0,418.837677,21901.0,1634.372968,6.367117,4.127692
3,16623.0,1420.447113,53.953410,9.945438,1853.860092,1779.0,381.310897,13419.0,1240.788459,4.978364,4.173613
4,14002.0,1189.451134,39.098277,6.343193,1546.289939,1818.0,358.848157,11891.0,1182.662251,3.797087,1.535186
5,6231.0,871.105619,28.416567,6.728655,1302.565248,363.0,164.042677,4982.0,676.295793,2.046770,1.499012
6,6754.0,1041.469635,inf,inf,1104.221088,608.0,225.308677,4675.0,702.936697,2.058680,1.021127
7,3248.0,683.357154,16.348509,7.069420,592.548718,487.0,184.669976,2389.0,526.624154,2.097745,0.932206


In [12]:
# Check that list indexing works
df[['B17001001', 'B17001002']].head()

,B17001001 Total,B17001001 Total,B17001002 Total Income in the past 12 months below poverty level,B17001002 Total Income in the past 12 months below poverty level
0,3092.0,3092.0,1092.0,1092.0
1,4517.0,4517.0,1276.0,1276.0
2,5874.0,5874.0,1281.0,1281.0
3,3912.0,3912.0,622.0,622.0
4,5902.0,5902.0,534.0,534.0
